In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [6]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=0.1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, hessian_approx=False)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.1482836753129959
epoch:  1, loss: 0.10641468316316605
epoch:  2, loss: 0.04400528594851494
epoch:  3, loss: 0.05026593059301376
epoch:  4, loss: 0.030253615230321884
epoch:  5, loss: 0.029083557426929474
epoch:  6, loss: 0.02852673828601837
epoch:  7, loss: 0.028158042579889297
epoch:  8, loss: 0.027902113273739815
epoch:  9, loss: 0.027728527784347534
epoch:  10, loss: 0.02749103493988514
epoch:  11, loss: 0.027259500697255135
epoch:  12, loss: 0.02708412893116474
epoch:  13, loss: 0.02694172039628029
epoch:  14, loss: 0.026780983433127403
epoch:  15, loss: 0.02665804512798786
epoch:  16, loss: 0.02654181979596615
epoch:  17, loss: 0.026432322338223457
epoch:  18, loss: 0.02635086327791214
epoch:  19, loss: 0.026273123919963837
epoch:  20, loss: 0.02622329443693161
epoch:  21, loss: 0.02617236040532589
epoch:  22, loss: 0.026114413514733315
epoch:  23, loss: 0.026037579402327538
epoch:  24, loss: 0.025961006060242653
epoch:  25, loss: 0.025877324864268303
epoch:  26

In [13]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=0.003, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, hessian_approx=True)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.1305222064256668
epoch:  1, loss: 0.11669120192527771
epoch:  2, loss: 0.10032882541418076
epoch:  3, loss: 0.08487207442522049
epoch:  4, loss: 0.07070424407720566
epoch:  5, loss: 0.05979813262820244
epoch:  6, loss: 0.05690736323595047
epoch:  7, loss: 0.06218424066901207
epoch:  8, loss: 0.05523960292339325
epoch:  9, loss: 0.0583680085837841
epoch:  10, loss: 0.05610703304409981
epoch:  11, loss: 0.05842607095837593
epoch:  12, loss: 0.05478939041495323
epoch:  13, loss: 0.05956759303808212
epoch:  14, loss: 0.05291202664375305
epoch:  15, loss: 0.061596136540174484
epoch:  16, loss: 0.05139549449086189
epoch:  17, loss: 0.05025597661733627
epoch:  18, loss: 0.06040771305561066
epoch:  19, loss: 0.04878214746713638
epoch:  20, loss: 0.0475023090839386
epoch:  21, loss: 0.06275629252195358
epoch:  22, loss: 0.04690377414226532
epoch:  23, loss: 0.06682387739419937
epoch:  24, loss: 0.048548270016908646
epoch:  25, loss: 0.05345550924539566
epoch:  26, loss: 0.046